<a href="https://colab.research.google.com/github/siribafna/cs_4372/blob/master/assignment_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [69]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tensorflow as tf
import keras.utils as utils
from tensorflow.keras.preprocessing import image_dataset_from_directory

# the following loading method was referenced from TensorFlow
location = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'
path = utils.get_file('cats_and_dogs.zip', origin=location, extract=True)
data_location = os.path.join(os.path.dirname(path), 'cats_and_dogs_filtered')

In [70]:
train_loc = os.path.join(data_location, 'train')
test_loc = os.path.join(data_location, 'validation')

In [71]:
train_dataset = image_dataset_from_directory(train_loc,
                                             shuffle=True,
                                             batch_size=32,
                                             image_size=(160, 160))

Found 2000 files belonging to 2 classes.


In [72]:
validation_dataset = image_dataset_from_directory(test_loc,
                                             shuffle=True,
                                             batch_size=32,
                                             image_size=(160, 160))

Found 1000 files belonging to 2 classes.


In [73]:
val_batches = tf.data.experimental.cardinality(validation_dataset)
test_dataset = validation_dataset.take(val_batches // 5)
validation_dataset = validation_dataset.skip(val_batches // 5)

In [106]:
inputs = tf.keras.Input(shape=(150, 150, 3))
x = data_augmentation(inputs)
x = preprocess_input(x)
x = base_model(x, training=False)
x = global_average_layer(x)
x = tf.keras.layers.Dropout(0.2)(x)
outputs = prediction_layer(x)
model = tf.keras.Model(inputs, outputs)

In [107]:
base_learning_rate = 0.0001
model.compile(optimizer=tf.keras.optimizers.Adam(lr=base_learning_rate),
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [108]:
history = model.fit(train_generator,
                    epochs=initial_epochs,
                    validation_data=test_generator)

Epoch 1/10
100/100 [==============================] - ETA: 0s - loss: 0.7106 - accuracy: 0.5035WARNING:tensorflow:Model was constructed with shape (None, 160, 160, 3) for input Tensor("input_8:0", shape=(None, 160, 160, 3), dtype=float32), but it was called on an input with incompatible shape (None, 150, 150, 3).


KeyboardInterrupt: ignored

In [113]:
from keras.preprocessing.image import ImageDataGenerator

train_img_data_gen = ImageDataGenerator(rescale=1./255,
rotation_range=35,
width_shift_range=0.4,
height_shift_range=0.14,
shear_range=0.142,
zoom_range=0.0001,
horizontal_flip=True)
)

test_img_data_gen = ImageDataGenerator(rescale=1./255) # as mentioned by the book, test would not be augmented

train_gen = train_img_data_gen.flow_from_directory(
train_loc,
target_size=(150, 150), #preprocessing, reducing size
batch_size=20,
class_mode='binary')

test_generator = test_datagen.flow_from_directory(
test_loc,
target_size=(150, 150),
batch_size=20,
class_mode='binary')

Found 2000 images belonging to 2 classes.
Found 1000 images belonging to 2 classes.


In [116]:
from keras.applications import VGG16
import keras.layers

test_model = VGG16(input_shape = (150, 150, 3), # Shape of our images
include_top = False, # Leave out the last fully connected layer
weights = 'imagenet')

for layer in test_model.layers:
    layer.trainable = False

x = tf.keras.layers.Flatten()(test_model.output)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

model = tf.keras.models.Model(test_model.input, x)

model.compile(optimizer = tf.keras.optimizers.RMSprop(lr=0.0001), loss = 'binary_crossentropy',metrics = ['acc'])

In [117]:
vgghist = model.fit(train_generator, validation_data = test_generator, steps_per_epoch = 100, epochs = 10)

Epoch 1/10
100/100 [==============================] - 672s 7s/step - loss: 0.4270 - acc: 0.7970 - val_loss: 0.3519 - val_acc: 0.8390
Epoch 2/10
100/100 [==============================] - 666s 7s/step - loss: 0.2766 - acc: 0.8775 - val_loss: 0.2721 - val_acc: 0.8800
Epoch 3/10
100/100 [==============================] - 670s 7s/step - loss: 0.2298 - acc: 0.9045 - val_loss: 0.2655 - val_acc: 0.8830
Epoch 4/10
100/100 [==============================] - 669s 7s/step - loss: 0.1805 - acc: 0.9210 - val_loss: 0.2683 - val_acc: 0.8750
Epoch 5/10
100/100 [==============================] - 664s 7s/step - loss: 0.1621 - acc: 0.9335 - val_loss: 0.3068 - val_acc: 0.8690
Epoch 6/10
100/100 [==============================] - ETA: 0s - loss: 0.1456 - acc: 0.9415

KeyboardInterrupt: ignored